In [27]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import spacy
from collections import defaultdict
import sys
sys.path.append('../../../../utils')
sys.path.append('..')
import gezi
from gezi import tqdm
from src.eval import calc_f1
from src.util import *
from src import config
from src.visualize import *
pd.set_option('display.float_format', lambda x: '%.02f' % x)

In [28]:
m = {}
df_gt = None

In [29]:
fold = 0
v = 16
v = f'{v}.{fold}'
mark = 'offline'
mns = [ 
   'tf.roberta-large.x',
   'tf.roberta-large.ts=left.x',
   'pt.roberta-large.x',
   'pt.roberta-large.ts=left.x', 
   'pt.large.max_len=1024.x',
   'pt.large.max_len=1024.ts=left.x',
   'pt.large.x',
   'pt.large.ts=left.x',
  ]
# weights = [1,1,1,1,1,1]
weights = None
classes = [
  'Claim', 'Evidence', 'Position', 'Concluding Statement', 'Lead', 'Counterclaim', 'Rebuttal'
]

In [30]:
model_dirs = [f'../working/{mark}/{v}/{mn}' for mn in mns]
mn = mns[0]

In [31]:
model_dir = f'../working/{mark}/{v}/{mn}'
argv = open(f'{model_dir}/command.txt').readline().strip().split()
FLAGS(argv)
# FLAGS.adjust_prob = True
# FLAGS.sent_min_len = 2
# FLAGS.sent_min_len2 = 6
# FLAGS.post_reduce_method = 'first'

['python', './main.py']

In [32]:
xs = [gezi.load(f'{model_dir}/valid.pkl') if model_dir not in m else m[model_dir] for model_dir in model_dirs]
for i, model_dir in enumerate(model_dirs):
  m[model_dir] = xs[i]

num_words = None
for x in xs: 
  if 'num_words' in x: 
    num_words = x['num_words']
    
x = xs[0]     
for x in xs:
  if 'num_words' not in x:
    x['num_words'] = num_words

In [33]:
if df_gt is None:
  df_gt = pd.read_feather(f'{FLAGS.idir}/train.fea')
  eval_ids = set(x['id'])
  df_gt = df_gt[df_gt['id'].isin(eval_ids)]
  df_gt = prepare(df_gt)
  df_gt_ = df_gt.copy()
  df_gt_['discourse_type'] = 'Claim'

  0%|          | 0/28809 [00:00<?, ?it/s]

  0%|          | 0/28808 [00:00<?, ?it/s]

In [34]:
for i, x in tqdm(enumerate(xs), total=len(xs)):
  if 'probs' not in x:
    convert_res(x)
x = ensemble_res(xs, weights)

  0%|          | 0/8 [00:00<?, ?it/s]

In [35]:
df_pred = get_preds(x)

get_preds:   0%|          | 0/3119 [00:00<?, ?it/s]

In [36]:
df_pred

,id,class,predictionstring
0,367111CA290A,Lead,4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 2...
1,367111CA290A,Position,74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 8...
2,367111CA290A,Evidence,93 94 95 96 97 98 99 100 101 102 103 104 105 1...
3,367111CA290A,Claim,254 255 256 257 258 259 260 261 262 263 264 26...
4,367111CA290A,Evidence,277 278 279 280 281 282 283 284 285 286 287 28...
...,...,...,...
27909,DBFD15D537CA,Claim,96 97 98 99 100 101 102 103 104 105 106 107 10...
27910,DBFD15D537CA,Evidence,114 115 116 117 118 119 120 121 122 123 124 12...
27911,DBFD15D537CA,Claim,179 180 181 182 183 184 185 186 187 188 189 19...
27912,DBFD15D537CA,Evidence,193 194 195 196 197 198 199 200 201 202 203 20...


In [37]:
df_pred = pd.merge(df_pred, df_gt[['id', 'text']].drop_duplicates(), on=['id'], how='left')
df_pred = prepare(df_pred)
df_pred.head()

  0%|          | 0/27914 [00:00<?, ?it/s]

  0%|          | 0/27913 [00:00<?, ?it/s]

,id,discourse_type,predictionstring,text,start,essay_len,essay_words,discourse_start,discourse_end,gap_before,gap_length,gap_end_length
0,00213CD75AC3,Lead,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,Cars have been an issue to our community for a...,0,2209,381,0,123,True,0.00,2086
1,00213CD75AC3,Claim,24 25 26 27 28 29 30 31 32 33 34 35 36 37,Cars have been an issue to our community for a...,24,2209,381,123,202,False,nan,2007
2,00213CD75AC3,Claim,38 39 40 41 42 43 44 45 46 47,Cars have been an issue to our community for a...,38,2209,381,202,278,False,nan,1931
3,00213CD75AC3,Position,48 49 50 51 52 53 54 55 56 57 58 59 60 61 62,Cars have been an issue to our community for a...,48,2209,381,278,354,False,nan,1855
4,00213CD75AC3,Evidence,63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 7...,Cars have been an issue to our community for a...,63,2209,381,354,693,False,nan,1516


In [38]:
res = {}
f1s = []
CLASSES = list(id2dis.values())[1:]
for c in CLASSES:
  ret = calc_f1(df_gt[df_gt['discourse_type'] == c],
               df_pred[df_pred['discourse_type'] == c], return_dict=True)
  res[c] = ret['f1']
  res[f'{c}/acc'] = ret['acc']
  res[f'{c}/recall'] = ret['recall']
  f1s.append(ret['f1'])
res['Overall'] = np.mean(f1s)
print(res['Overall'])
res

0.6961013696135806


{'Claim': 0.6648112167200546,
 'Claim/acc': 0.7054496823804748,
 'Claim/recall': 0.6285998013902682,
 'Evidence': 0.761723298401609,
 'Evidence/acc': 0.7418556701030928,
 'Evidence/recall': 0.7826843593648032,
 'Position': 0.7159261108337494,
 'Position/acc': 0.7348821318756406,
 'Position/recall': 0.6979234263465283,
 'Concluding Statement': 0.8579205225911812,
 'Concluding Statement/acc': 0.8341566690190544,
 'Concluding Statement/recall': 0.8830780724691819,
 'Lead': 0.8176299815254685,
 'Lead/acc': 0.8055122204888195,
 'Lead/recall': 0.830117899249732,
 'Counterclaim': 0.5652173913043478,
 'Counterclaim/acc': 0.6183932346723044,
 'Counterclaim/recall': 0.5204626334519573,
 'Rebuttal': 0.4894810659186536,
 'Rebuttal/acc': 0.5601926163723917,
 'Rebuttal/recall': 0.43462017434620176,
 'Overall': 0.6961013696135806}

In [39]:
!ls ../working/offline/7.0/pt.large.x1

ls: cannot access '../working/offline/7.0/pt.large.x1': No such file or directory


In [40]:
eval_ids = list(eval_ids)
f1s = []
for id in tqdm(eval_ids):
  f1s.append(calc_f1(df_gt[df_gt['id'] == id], df_pred[df_pred['id'] == id]))

  0%|          | 0/3119 [00:00<?, ?it/s]

In [47]:
df = pd.DataFrame(
  {
    'id': eval_ids,
    'f1': f1s
  }
)
df

,id,f1
0,90631D30D477,0.40
1,196E73AAF08F,0.77
2,FF01B32BAC3E,0.80
3,9215ABF587EB,0.71
4,05FFBFA9B3BA,0.86
...,...,...
3114,0C24337CD49E,0.22
3115,CE28B6B31CCE,0.71
3116,A2BCD0012914,0.83
3117,30E611671A72,0.92


In [48]:
df.describe()

,f1
count,3119.00
mean,0.69
std,0.22
min,0.00
25%,0.56
50%,0.73
75%,0.86
max,1.00


In [49]:
df.sort_values(['f1'], ascending=True, inplace=True)

In [50]:
df.id.values

array(['8105CDA99A3C', 'F744ABAAC3FB', 'A1F48D16C67B', ...,
       '2D36D365D242', '0A3818A64CF5', 'E0ACC902891D'], dtype=object)

In [51]:
def visualize(essay):
  f1 = calc_f1(
      df_gt[df_gt['id'] == essay],
      df_pred[df_pred['id'] == essay].rename({'discourse_type': 'class'},
                                             axis=1))
  f1_binary = calc_f1(
      df_gt_[df_gt_['id'] == essay],
      df_pred_[df_pred_['id'] == essay].rename({'discourse_type': 'class'},
                                               axis=1))
  print(essay, 'f1:', f1, 'f1_binary:', f1_binary)
  visulize_df(df_gt, essay)
  visulize_df(df_pred, essay)

In [52]:
visualize(df.id.values[0])

NameError: name 'df_pred_' is not defined

In [ ]:
visualize(df.id.values[1])

In [ ]:
visualize(df.id.values[2])

In [ ]:
visualize(df.id.values[3])

In [ ]:
visualize(df.id.values[5])

In [ ]:
visualize(df.id.values[random.randint(0, len(eval_ids) - 1)])

In [ ]:
visualize(df.id.values[-1])

In [ ]:
visualize('193591CD2E86')